# Analyse Exploratoire

Ce notebook renferme toute l'analyse exploratoire concernant le projet "Segmentez des clients d'un site e-commerce".

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def translate_product_category(data_products):
    mapping_names = pd.read_csv("product_category_name_translation.csv", encoding='utf-8', sep=',', index_col="product_category_name").to_dict()
    if 'product_category_name' in df.columns:
        data_products.replace({"product_category_name": mapping_names['product_category_name_english']}, inplace=True)

# I) Création des datasets brut

Les données sont dispersées en 8 fichiers. Le fichier sur la géolocalisation ne nous sera pas utile pour notre étude. En effet, toutes les coordonnées pointent vers le Brésil. Nous allons donc charger les autres fichiers et tous les fusionner pour obtenir notre dataset brut.

In [37]:
def load_data():
    all_data = {}
    data_products = pd.read_csv(
        "input/olist_products_dataset.csv", encoding='utf-8', sep=',')
        
    data_customers = pd.read_csv(
        "input/olist_customers_dataset.csv", encoding='utf-8', sep=',')
    
    data_items = pd.read_csv(
        "input/olist_order_items_dataset.csv", encoding='utf-8', sep=',',
        parse_dates=["shipping_limit_date"])
    
    order_date_col = [
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date']
    data_orders = pd.read_csv(
        "input/olist_orders_dataset.csv", encoding='utf-8', sep=',',
        parse_dates=order_date_col)
    
    review_date_col = [
        "review_creation_date",
        "review_answer_timestamp"]
    data_reviews = pd.read_csv(
        "input/olist_order_reviews_dataset.csv", encoding='utf-8', sep=',',
        parse_dates=review_date_col)
    
    data_payments = pd.read_csv(
        "input/olist_order_payments_dataset.csv", encoding='utf-8', sep=',')
    
    data_sellers = pd.read_csv(
        "input/olist_sellers_dataset.csv", encoding='utf-8', sep=',')
    
    all_data["products"] = data_products
    all_data["customers"] = data_customers
    all_data["items"] = data_items
    all_data["orders"] = data_orders
    all_data["payments"] = data_payments
    all_data["reviews"] = data_reviews
    all_data["sellers"] = data_sellers
    
    for name, dataset in all_data.items():
        print("There are", dataset.shape[0], "", name, "characterised by", dataset.shape[1], "columns.")
    
    return all_data

In [38]:
raw_data = load_data()

There are 32951  products characterised by 9 columns.
There are 99441  customers characterised by 5 columns.
There are 112650  items characterised by 7 columns.
There are 99441  orders characterised by 8 columns.
There are 103886  payments characterised by 5 columns.
There are 99224  reviews characterised by 7 columns.
There are 3095  sellers characterised by 4 columns.


In [39]:
def merge_all_data(raw_data):
    master_df = pd.merge(raw_data["products"], raw_data["items"], on="product_id", how="outer")
    master_df = pd.merge(master_df, raw_data["orders"], on="order_id", how="outer")
    master_df = pd.merge(master_df, raw_data["payments"], on="order_id", how="outer")
    master_df = pd.merge(master_df, raw_data["reviews"], on="order_id", how="outer")
    master_df = pd.merge(master_df, raw_data["customers"], on="customer_id", how="outer")
    print("There are", master_df.shape[0], "lines characterised by", master_df.shape[1], "columns.")

In [40]:
data = merge_all_data(raw_data)

There are 119143 lines characterised by 36 columns.


In [32]:
master_df.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,order_id,...,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,e17e4f88e31525f7deef66779844ddce,...,d71da8fd8c6e3adef26be965f065b8a1,5.0,Vale a pena,Super adooorei o delineador ele é bem preto e ...,2018-04-28,2018-05-04 00:55:02,b1a1199364a4a7fe27c4486ab63f550d,13848,mogi-guacu,SP
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,5236307716393b7114b53ee991f36956,...,19866247db89845ef558ef19a6c26fcd,5.0,NaN,Excelente apontador! Em 3 segundos uma ponta p...,2018-02-17,2018-02-18 23:24:14,4b86049cb99e4aa774031daa9cd18f18,20551,rio de janeiro,RJ
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,01f66e58769f84129811d43eefd187fb,...,32921b4e09c9a19d61fe46ce04231b84,5.0,NaN,NaN,2018-07-14,2018-07-16 16:20:04,f63805d9c7edb84d92413af34b86a39c,5821,sao paulo,SP
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,143d00a4f2dde4e0364ee1821577adb3,...,70949d129965fb1ee32aaccec720a19e,1.0,produto divergente,modelo comprado: travesseiro memoria carinho B...,2018-08-07,2018-08-08 11:53:04,619e926d09b26efbd5180368b1ddc874,2018,sao paulo,SP
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,86cafb8794cb99a9b1b77fc8e48fbbbb,...,74c9c7d0bc153bbbc92e4c1597b8ba55,5.0,NaN,NaN,2018-04-17,2018-04-18 04:29:47,ad353b4fb0e294adc4eda48af73e68a6,5835,sao paulo,SP


In [33]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119143 entries, 0 to 119142
Data columns (total 36 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   product_id                     118310 non-null  object        
 1   product_category_name          116601 non-null  object        
 2   product_name_lenght            116601 non-null  float64       
 3   product_description_lenght     116601 non-null  float64       
 4   product_photos_qty             116601 non-null  float64       
 5   product_weight_g               118290 non-null  float64       
 6   product_length_cm              118290 non-null  float64       
 7   product_height_cm              118290 non-null  float64       
 8   product_width_cm               118290 non-null  float64       
 9   order_id                       119143 non-null  object        
 10  order_item_id                  118310 non-null  float64       
 11  

In [34]:
master_df.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,order_item_id,price,freight_value,payment_sequential,payment_installments,payment_value,review_score,customer_zip_code_prefix
count,116601.000000,116601.000000,116601.000000,118290.000000,118290.000000,118290.000000,118290.000000,118310.000000,118310.000000,118310.000000,119140.000000,119140.000000,119140.000000,118146.000000,119143.000000
mean,48.767498,785.967822,2.205161,2112.250740,30.265145,16.619706,23.074799,1.196543,120.646603,20.032387,1.094737,2.941246,172.735135,4.015582,35033.451298
std,10.033540,652.584121,1.717452,3786.695111,16.189367,13.453584,11.749139,0.699489,184.109691,15.836850,0.730141,2.777848,267.776077,1.400436,29823.198969
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000,1.000000,0.850000,0.000000,1.000000,0.000000,0.000000,1.000000,1003.000000
25%,42.000000,346.000000,1.000000,300.000000,18.000000,8.000000,15.000000,1.000000,39.900000,13.080000,1.000000,1.000000,60.850000,4.000000,11250.000000
50%,52.000000,600.000000,1.000000,700.000000,25.000000,13.000000,20.000000,1.000000,74.900000,16.280000,1.000000,2.000000,108.160000,5.000000,24240.000000
75%,57.000000,983.000000,3.000000,1800.000000,38.000000,20.000000,30.000000,1.000000,134.900000,21.180000,1.000000,4.000000,189.240000,5.000000,58475.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000,21.000000,6735.000000,409.680000,29.000000,24.000000,13664.080000,5.000000,99990.000000


In [36]:
valeurs_manquantes = round(master_df.isnull().mean()*100, 4).sort_values(ascending=False)
missing_val_df = pd.DataFrame(valeurs_manquantes, columns = ["%"])
missing_val_df.head(50)

,%
review_comment_title,88.2586
review_comment_message,57.8280
order_delivered_customer_date,2.8713
product_name_lenght,2.1336
product_description_lenght,2.1336
product_photos_qty,2.1336
product_category_name,2.1336
order_delivered_carrier_date,1.7508
review_score,0.8368
review_answer_timestamp,0.8368


## 2) Dataset brut Clients

# II) Préparation des dataset d'études

# III) Compréhension des datasets d'étude